In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
credit_card_data = pd.read_csv('/content/creditcard.csv')

In [ ]:
credit_card_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


## EDA

In [ ]:
credit_card_data.shape

(284807, 31)

In [ ]:
credit_card_data.dtypes

,0
Time,float64
V1,float64
V2,float64
V3,float64
V4,float64
V5,float64
V6,float64
V7,float64
V8,float64
V9,float64


In [ ]:
credit_card_data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [ ]:
credit_card_data['Class'].value_counts()

,count
Class,
0,284315
1,492


Highly Imbalanced target classes, needs SMOTE Technique

In [ ]:
X = credit_card_data.drop(['Class', 'Time'], axis=1)
y = credit_card_data['Class']

In [ ]:
X

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,      # 80% train, 20% test
    stratify=y,          # Keep same fraud ratio in both sets
    random_state=42
)

In [ ]:
print("Original split:")
print(f"Train - Fraud: {sum(y_train)} | Legitimate: {len(y_train) - sum(y_train)}")
print(f"Test  - Fraud: {sum(y_test)} | Legitimate: {len(y_test) - sum(y_test)}")

Original split:
Train - Fraud: 394 | Legitimate: 227451
Test  - Fraud: 98 | Legitimate: 56864


## SMOTE - Oversampling

In [ ]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [ ]:
print("\nAfter SMOTE on training data:")
print(f"Train - Fraud: {sum(y_train_balanced)} | Legitimate: {len(y_train_balanced) - sum(y_train_balanced)}")
print(f"Test  - Fraud: {sum(y_test)} | Legitimate: {len(y_test) - sum(y_test)}")


After SMOTE on training data:
Train - Fraud: 227451 | Legitimate: 227451
Test  - Fraud: 98 | Legitimate: 56864


## Scale the data
#### (As Amount Feature range is much higher than other columns)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test)

## Logistic Regression

In [ ]:
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train_balanced)

LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
X_train_prediction = lr.predict(X_train_scaled)
training_data_accuracy = accuracy_score(X_train_prediction, y_train_balanced)

In [ ]:
training_data_accuracy

0.9639438824186308

In [ ]:
X_test_prediction = lr.predict(X_test_scaled)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [ ]:
test_data_accuracy

0.9811979916435518

In [ ]:
cm_lr = confusion_matrix(y_test, X_test_prediction)
print(cm_lr)

[[55802  1062]
 [    9    89]]


In [ ]:
f1_lr = f1_score(y_test, X_test_prediction)
print(f1_lr)

0.14251401120896717


In [ ]:
recall_lr = recall_score(y_test, X_test_prediction)
print(recall_lr)

0.9081632653061225


In [ ]:
precision_lr = precision_score(y_test, X_test_prediction)
print(precision_lr)

0.07732406602953953


## Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_scaled, y_train_balanced)

RandomForestClassifier(random_state=42)

In [ ]:
rf_X_test_prediction = rf.predict(X_test_scaled)

In [ ]:
rf_test_data_accuracy = accuracy_score(rf_X_test_prediction, y_test)

In [ ]:
cm_rf = confusion_matrix(y_test, rf_X_test_prediction)
print(cm_rf)

[[56849    15]
 [   15    83]]


In [ ]:
f1_rf = f1_score(y_test, rf_X_test_prediction)
print(f1_rf)

0.8469387755102041


In [ ]:
recall_rf = recall_score(y_test, rf_X_test_prediction)
print(recall_rf)

0.8469387755102041


In [ ]:
precision_rf = precision_score(y_test, rf_X_test_prediction)
print(precision_rf)

0.8469387755102041


## XGBoost

In [ ]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train_scaled, y_train_balanced)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
xgb_X_test_prediction = xgb.predict(X_test_scaled)

In [ ]:
xgb_test_data_accuracy = accuracy_score(xgb_X_test_prediction, y_test)

In [ ]:
xgb_test_data_accuracy

0.9994733330992591

In [ ]:
cm_xgb = confusion_matrix(y_test, xgb_X_test_prediction)
print(cm_xgb)

[[56847    17]
 [   13    85]]


In [ ]:
f1_xgb = f1_score(y_test, xgb_X_test_prediction)
print(f1_xgb)

0.85


In [ ]:
recall_xgb = recall_score(y_test, xgb_X_test_prediction)
print(recall_xgb)

0.8673469387755102


In [ ]:
precision_xgb = precision_score(y_test, xgb_X_test_prediction)
print(precision_xgb)

0.8333333333333334


## Ensemble Stacking

In [ ]:
base_models = [
    ('lr', LogisticRegression(max_iter=1000, random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=100, max_depth=10,
                                   min_samples_leaf=4, random_state=42)),
    ('xgb', XGBClassifier(scale_pos_weight=1, learning_rate=0.05,
                          n_estimators=100, subsample=0.8, random_state=42))
]

In [ ]:
def get_models():
  lr_s = LogisticRegression(max_iter=1000, random_state=42)
  rf_s = RandomForestClassifier(random_state=42)
  xgb_s = XGBClassifier(random_state=42)
  return lr_s, rf_s, xgb_s

In [ ]:
lr_model, rf_model, xgb_model = get_models()
lr_model.fit(X_train_scaled, y_train_balanced)
rf_model.fit(X_train_scaled, y_train_balanced)
xgb_model.fit(X_train_scaled, y_train_balanced)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
# --- Test Ensemble ---
lr_test = lr_model.predict_proba(X_test_scaled)
rf_test = rf_model.predict_proba(X_test_scaled)
xgb_test = xgb_model.predict_proba(X_test_scaled)

In [ ]:
# --- weights for Ensemble ---
w1, w2 = 0.33, 0.33
w3 = 1 - w1 - w2

In [ ]:
ensemble_test = (w1 * lr_test + w2 * rf_test + w3 * xgb_test)
ensemble_test_pred = np.argmax(ensemble_test, axis=1)

In [ ]:
print("Accuracy:", accuracy_score(y_test, ensemble_test_pred))
print("F1:", f1_score(y_test, ensemble_test_pred))
print("Recall ensemble:", recall_score(y_test, ensemble_test_pred))
print("Precision ensemble:", precision_score(y_test, ensemble_test_pred))

Accuracy: 0.9993679997191109
F1: 0.8285714285714286
Recall ensemble: 0.8877551020408163
Precision ensemble: 0.7767857142857143


In [ ]:
cm_ensemble = confusion_matrix(y_test, ensemble_test_pred)
print(cm_ensemble)

[[56839    25]
 [   11    87]]


In [ ]:
# --- weights for Ensemble ---
w1, w2 = 0.35, 0.35
w3 = 1 - w1 - w2

In [ ]:
ensemble_test = (w1 * lr_test + w2 * rf_test + w3 * xgb_test)
ensemble_test_pred = np.argmax(ensemble_test, axis=1)

In [ ]:
print("Accuracy:", accuracy_score(y_test, ensemble_test_pred))
print("F1:", f1_score(y_test, ensemble_test_pred))
print("Recall ensemble:", recall_score(y_test, ensemble_test_pred))
print("Precision ensemble:", precision_score(y_test, ensemble_test_pred))

Accuracy: 0.9992977774656788
F1: 0.8130841121495327
Recall ensemble: 0.8877551020408163
Precision ensemble: 0.75


In [ ]:
cm_ensemble = confusion_matrix(y_test, ensemble_test_pred)
print(cm_ensemble)

[[56835    29]
 [   11    87]]
